In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader

# Build the transform for the train dataset and test dataset.
train_transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize([224, 224]),
     torchvision.transforms.ToTensor()])

test_transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize([224, 224]), torchvision.transforms.ToTensor()])

# Build the train dataset and test dataset
train_dataset = torchvision.datasets.Flowers102(root='./dataset', split='train', transform=train_transform,
                                                download=True)
test_dataset = torchvision.datasets.Flowers102(root='./dataset', split='val', transform=test_transform,
                                               download=True)

# Build the train dataloader and test dataloader use the two datasets.
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True, drop_last=True)

# Build the Convolution Neural Network
# But we can build the resnet18 easily by the torchvision:
model = torchvision.models.resnet18(torchvision.models.ResNet18_Weights)

# Build the loss function and optimizer
# For the Image Classification task, we use the cross-entropy loss function as the loss function and use the SGD(Stochastic Gradient Descent) as the optimizer.


loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)


# Train the model
def train():
    all_loss = 0.0
    n = 0
    for data in train_dataloader:
        n = n + 1
        optimizer.zero_grad()  # Set 0 into grads of optimizer
        image, target = data  # Fetch the data and target
        output = model(image)  # Forward
        loss = loss_function(output, target)  # Calculate the loss
        loss.backward()  # Backward
        optimizer.step()  # Optimizer works
        all_loss += loss.item()
        print('Train process: %.3f of this epoch, loss : %.2f ' % (n / len(train_dataloader), loss.item()))
    return all_loss / len(train_dataloader)  # return the loss


# Test the model
def test():
    model.eval()  # set the model into the evaluation mode, stopping Backward.
    all_acc = 0.0
    n = 0
    for data in test_dataloader:
        n = n + 1
        image, target = data  # Fetch the data
        output = model(image)  # Forward
        print('Test process: %.2f of this epoch' % (n / len(test_dataloader)))
        all_acc += torch.eq(torch.argmax(output, dim=-1), target).float().mean()  # Partial accuary
    model.train()  # set the model into training mode
    return all_acc / len(test_dataloader)


def main():
    best_acc = 0.0
    for i in range(15):  # train for 15 epochs
        # best accuary
        loss = train()
        acc = test()
        print(f"epoch: {i}, loss: {loss}, accuary: {acc}")
        if acc > best_acc:
            torch.save(model, 'best.pth')  # save the best model
            best_acc = acc


if __name__ == '__main__':
    main()
